In [21]:
import tskit
import msprime
import numpy as np
import pyslim
import networkx as nx
from IPython.display import SVG
import math

In [4]:
# Mike's function for calculating effective dispersal rate from a tree sequence
def effective_dispersal(ts, max_edge_length=math.inf):
    """
    Returns the mean standardized squared distance (in x and y) between all
    parent-child relationships in the recorded tree sequence separated
    by a time interval less than or equal to `max_edge_length`. This is an
    estimator for the variance of a Gaussian dispersal kernel.
    """
    assert type(ts) == tskit.trees.TreeSequence
    nx = ny = 0
    mx = my = 0
    for edge in ts.edges():
        parent_id = edge.parent
        child_id = edge.child
        edge_length = ts.node(parent_id).time - ts.node(child_id).time
        if edge_length <= max_edge_length:
            parent_individual_id = ts.node(parent_id).individual
            child_individual_id = ts.node(child_id).individual
            if parent_individual_id and child_individual_id:
                parent_loc = ts.individual(parent_individual_id).location
                child_loc = ts.individual(child_individual_id).location
                if len(parent_loc) >= 2 and len(child_loc) >= 2:
                    dx = (parent_loc[0] - child_loc[0]) / math.sqrt(edge_length)
                    dy = (parent_loc[1] - child_loc[1]) / math.sqrt(edge_length)
                    if nx:
                        nx += 1
                        mx += (dx*dx - mx) / nx
                    else:
                        nx = 1
                        mx = dx*dx
                    if ny:
                        ny += 1
                        my += (dy*dy - my) / ny
                    else:
                        ny = 1
                        my = dy*dy
    return [mx, my]

def min(a, b):
     
    if a <= b:
        return a
    else:
        return b

In [35]:
### PROCESSING VARIABLES ###
tree_separation = 50

ts_upload_path = '/Users/alexlewanski/Documents/michigan_state/research/location_imputation/simulation_output/'
output_file_path = '/Users/alexlewanski/Documents/michigan_state/research/location_imputation/simulation_output/'
output_tree_path = '/Users/alexlewanski/Documents/michigan_state/research/location_imputation/simulation_output/tree_dir/'

In [36]:
### LOAD, RECAPITATE, AND SIMPLIFY TREE SEQUENCE ### 
spatial_ts = tskit.load(ts_upload_path + "example_sim.trees")


spatial_ts1 = pyslim.recapitate(
    spatial_ts,
    recombination_rate = 1e-8,
    ancestral_Ne=500,
    random_seed=71110)

spatial_ts2 = spatial_ts1.simplify()

In [37]:
sample_indiv_array = np.unique(spatial_ts2.nodes_individual[np.where(spatial_ts2.nodes_flags == 1)])
#subsample_sample = np.random.choice(sample_indiv_array, size=n_subsamp, replace=False)
#node_array = [[np.where(ts.nodes_individual == INDIV)[0]][0] for INDIV in subsample_sample]

subsample_sample = np.random.choice(sample_indiv_array, size=80, replace=False)
subsamp_node_list = [spatial_ts2.individual(x).nodes for x in subsample_sample]
spatial_ts2_reduced = spatial_ts2.simplify(np.concatenate(subsamp_node_list))


#example of clustered sampling
'''
indivs_in_sampled_locs = [
    INDIV for INDIV in sample_indiv_array
    if (
        (0 <= spatial_ts2.individual(INDIV).location[0] <= 15 and 5 <= spatial_ts2.individual(INDIV).location[1] <= 25)
        or
        (45 <= spatial_ts2.individual(INDIV).location[0] <= 60 and 45 <= spatial_ts2.individual(INDIV).location[1] <= 60)
        or
        (80 <= spatial_ts2.individual(INDIV).location[0] <= 95 and 7 <= spatial_ts2.individual(INDIV).location[1] <= 18)
        or
        (35 <= spatial_ts2.individual(INDIV).location[0] <= 45 and 68 <= spatial_ts2.individual(INDIV).location[1] <= 75)
        or
        (0 <= spatial_ts2.individual(INDIV).location[0] <= 15 and 70 <= spatial_ts2.individual(INDIV).location[1] <= 85)
        or
        (60 <= spatial_ts2.individual(INDIV).location[0] <= 80 and 90 <= spatial_ts2.individual(INDIV).location[1] <= 100)
    )
]


indivs_in_sampled_locs_subsample = np.random.choice(indivs_in_sampled_locs, size=80, replace=False)

extra_indivs = np.random.choice(sample_indiv_array, size=8, replace=False)

combined = np.unique(np.concatenate((indivs_in_sampled_locs_subsample, extra_indivs)))

subsamp_node_list = [spatial_ts2.individual(x).nodes for x in combined]

spatial_ts2_reduced = spatial_ts2.simplify(np.concatenate(subsamp_node_list))
'''

'\nindivs_in_sampled_locs = [\n    INDIV for INDIV in sample_indiv_array\n    if (\n        (0 <= spatial_ts2.individual(INDIV).location[0] <= 15 and 5 <= spatial_ts2.individual(INDIV).location[1] <= 25)\n        or\n        (45 <= spatial_ts2.individual(INDIV).location[0] <= 60 and 45 <= spatial_ts2.individual(INDIV).location[1] <= 60)\n        or\n        (80 <= spatial_ts2.individual(INDIV).location[0] <= 95 and 7 <= spatial_ts2.individual(INDIV).location[1] <= 18)\n        or\n        (35 <= spatial_ts2.individual(INDIV).location[0] <= 45 and 68 <= spatial_ts2.individual(INDIV).location[1] <= 75)\n        or\n        (0 <= spatial_ts2.individual(INDIV).location[0] <= 15 and 70 <= spatial_ts2.individual(INDIV).location[1] <= 85)\n        or\n        (60 <= spatial_ts2.individual(INDIV).location[0] <= 80 and 90 <= spatial_ts2.individual(INDIV).location[1] <= 100)\n    )\n]\n\n\nindivs_in_sampled_locs_subsample = np.random.choice(indivs_in_sampled_locs, size=80, replace=False)\n\nextr

In [38]:
### EXTRACTING INFO ABOUT INDIVIDUALS (COORDINATES AND NODES ASSOCIATED WITH EACH INDIV) ###
sample_list_rename = np.unique(spatial_ts2_reduced.nodes_individual[np.where(spatial_ts2_reduced.nodes_flags == 1)])

indiv_list = []
node_list = []
x_list = []
y_list = []

for INDIV in sample_list_rename:
    for NODE in spatial_ts2_reduced.individual(INDIV).nodes:
        indiv_list.append(INDIV)
        node_list.append(NODE)
        x_list.append(spatial_ts2_reduced.individual(INDIV).location[0])
        y_list.append(spatial_ts2_reduced.individual(INDIV).location[1])

column_names = ['indiv', 'node', 'x', 'y']

with open(output_file_path + 'indiv_node_info.txt', 'w') as file:
    # Write the rows, aligning each list as a column
    file.write("\t".join(column_names) + "\n")
    for i,x in enumerate(indiv_list):
            row = f"{indiv_list[i]}\t{node_list[i]}\t{x_list[i]}\t{y_list[i]}\n"
            file.write(row)

In [39]:
### EXTRACTING INFO ABOUT THE TREES (COORDINATES OF ROOT AND GENOME SPAN) ###
tree_index_list = []
tree_position_list = []
node_list = []
x_list = []
y_list = []
span_list = []

for i,tree_pos in enumerate(range(0, spatial_ts2_reduced.num_trees, tree_separation)):
    tree_index_list.append(i)
    tree_position_list.append(tree_pos)
    x_list.append(spatial_ts2_reduced.individual(spatial_ts2_reduced.node(spatial_ts2_reduced.at_index(tree_pos).root).individual).location[0])
    y_list.append(spatial_ts2_reduced.individual(spatial_ts2_reduced.node(spatial_ts2_reduced.at_index(tree_pos).root).individual).location[1])
    node_list.append(spatial_ts2_reduced.node(spatial_ts2_reduced.at_index(tree_pos).root).id)
    span_list.append(spatial_ts2_reduced.at_index(tree_pos).span)
    
column_names = ['tree_index', 'tree_position', 'node', 'x', 'y', 'span']



with open(output_file_path + 'tree_position_info.txt', 'w') as file:
    # Write the rows, aligning each list as a column
    file.write("\t".join(column_names) + "\n")
    for i,x in enumerate(tree_index_list):
            row = f"{tree_index_list[i]}\t{tree_position_list[i]}\t{node_list[i]}\t{x_list[i]}\t{y_list[i]}\t{span_list[i]}\n"
            file.write(row)


In [40]:
### OUTPUTTING TREES AS NEWICKS ###
for TREE in range(0, spatial_ts2_reduced.num_trees, tree_separation):
    with open(output_tree_path +  "spatial_sim_test_newick" + str(TREE) + ".nwk", "w") as f:
        f.write(spatial_ts2_reduced.at_index(TREE).as_newick())
        print('exported' + str(TREE))

exported0
exported50
exported100
exported150
exported200
exported250
exported300
exported350
exported400
exported450
exported500
exported550
exported600
exported650
exported700
exported750
exported800
exported850
exported900
exported950
exported1000
exported1050
exported1100
exported1150
exported1200
exported1250
exported1300
exported1350
exported1400
exported1450
exported1500
exported1550
exported1600
exported1650
exported1700
exported1750
exported1800
exported1850
exported1900
exported1950
exported2000
exported2050
exported2100
exported2150
exported2200
exported2250
exported2300
exported2350
exported2400
exported2450
exported2500
exported2550
exported2600
exported2650
exported2700
exported2750
exported2800
exported2850
exported2900
exported2950
exported3000
exported3050
exported3100
exported3150
exported3200
exported3250
exported3300
exported3350
exported3400
exported3450
exported3500
exported3550
exported3600
exported3650
exported3700
exported3750
exported3800
exported3850
exported3

In [29]:
effective_dispersal(spatial_ts2_reduced, max_edge_length=2500)

[4.054292124406332, 4.0762866116524155]